In [1]:
!pip install -q torch opencv-python pandas
!pip install -q kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 873.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 50.9 MB/s eta 0:00:00


In [2]:
!kaggle datasets download -d ayushspai/sportsmot -q
!unzip -q sportsmot.zip -d SportsMOT
!rm sportsmot.zip
# !mv /content/SportsMOT/sportsmot_publish/dataset/val/* /content/SportsMOT/sportsmot_publish/dataset/train > /dev/null 2>&1

Dataset URL: https://www.kaggle.com/datasets/ayushspai/sportsmot
License(s): MIT


In [10]:
import os
import cv2
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.transforms import Compose, ToTensor, Normalize, RandomHorizontalFlip, ColorJitter

import matplotlib.pyplot as plt
import matplotlib.patches as patches

from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


## Dataset

In [4]:
def load_football_sequences(splits_dir):
    football_file = os.path.join(splits_dir, "football.txt")
    with open(football_file, "r") as f:
        all_sequences = f.read().splitlines()

    return all_sequences

## Object Detection

In [5]:
!pip install -q ultralytics matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 23.0 MB/s eta 0:00:00


In [7]:
def convert_to_yolo_format(root_dir, mot_dir, split):
    os.makedirs(os.path.join(root_dir, "dataset", "images", split), exist_ok=True)
    os.makedirs(os.path.join(root_dir, "dataset", "labels", split), exist_ok=True)

    splits_dir = os.path.join(mot_dir, "splits_txt")
    football_sequences = load_football_sequences(splits_dir)

    for seq in football_sequences:
        seq_path = os.path.join(mot_dir, "dataset", split, seq)
        gt_file = os.path.join(seq_path, "gt", "gt.txt")

        if not os.path.exists(gt_file):
            continue

        df = pd.read_csv(gt_file, header=None)
        df.columns = [
            "frame_id", "object_id", "x", "y", "w", "h",
            "conf", "class_id", "visibility"
        ]

        for frame_id, group in df.groupby("frame_id"):
            img_path = os.path.join(seq_path, "img1", f"{frame_id:06d}.jpg")
            if not os.path.exists(img_path):
                continue

            img = cv2.imread(img_path)
            img_name = f"{seq}_{frame_id:06d}.jpg"
            cv2.imwrite(os.path.join(root_dir, "dataset", "images", split, img_name), img)

            label_file = os.path.join(root_dir, "dataset", "labels", split, f"{seq}_{frame_id:06d}.txt")
            with open(label_file, "w") as f:
                for _, row in group.iterrows():
                    x_center = (row["x"] + row["w"] / 2) / img.shape[1]
                    y_center = (row["y"] + row["h"] / 2) / img.shape[0]
                    width = row["w"] / img.shape[1]
                    height = row["h"] / img.shape[0]
                    f.write(f"{int(row['class_id'])} {x_center} {y_center} {width} {height}\n")

convert_to_yolo_format("/content", "/content/SportsMOT/sportsmot_publish", "train")
convert_to_yolo_format("/content", "/content/SportsMOT/sportsmot_publish", "val")
convert_to_yolo_format("/content", "/content/SportsMOT/sportsmot_publish", "test")

In [12]:
import os

def create_yaml_file(dataset_dir, class_names):
    yaml_content = f"""
path: {dataset_dir}
train: images/train
val: images/val
test: images/test

nc: {len(class_names)}
names: {class_names}
"""
    yaml_path = os.path.join(dataset_dir, "dataset.yaml")
    with open(yaml_path, "w") as f:
        f.write(yaml_content)

    print(f"YAML file created at: {yaml_path}")

dataset_dir = "/content/dataset"
class_names = ['player', 'referee', 'ball', 'other']

create_yaml_file(dataset_dir, class_names)

YAML file created at: /content/dataset/dataset.yaml


In [13]:
model = YOLO("yolov8n.pt")

results = model.train(
    data="/content/dataset/dataset.yaml",
    epochs=50,
    imgsz=720,
    batch=16,
    name="yolov8_football"
)

Ultralytics 8.3.70 🚀 Python-3.11.11 torch-2.5.1+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/dataset/dataset.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=720, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8_football2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True,

100%|██████████| 755k/755k [00:00<00:00, 22.9MB/s]


Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/dataset/labels/train... 20171 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20171/20171 [00:37<00:00, 533.10it/s] 

train: WARNING ⚠️ /content/dataset/images/train/v_i2_L4qquVg0_c006_000456.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/dataset/images/train/v_i2_L4qquVg0_c006_000462.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/dataset/images/train/v_i2_L4qquVg0_c006_000475.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/dataset/images/train/v_i2_L4qquVg0_c006_000482.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/dataset/images/train/v_i2_L4qquVg0_c006_000485.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/dataset/images/train/v_i2_L4qquVg0_c006_000492.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/dataset/images/train/v_i2_L4qquVg0_c006_000494.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/dataset/images/train/v_i2_L4qquVg0_c006_000499.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/dataset/images/train/v_i2_L4qquVg0_c006_000500.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/dataset/images/trai

train: New cache created: /content/dataset/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.2 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


FileNotFoundError: [34m[1mval: [0mError loading data from /content/dataset/images/val
See https://docs.ultralytics.com/datasets for dataset formatting guidance.

In [ ]:
logs = pd.read_csv("/path/to/training/logs/results.csv")

plt.figure(figsize=(10, 5))
plt.plot(logs["epoch"], logs["train/box_loss"], label="Train Box Loss")
plt.plot(logs["epoch"], logs["val/box_loss"], label="Validation Box Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.show()

plt.figure(figsize=(10, 5))
plt.plot(logs["epoch"], logs["metrics/mAP_0.5"], label="mAP@0.5")
plt.plot(logs["epoch"], logs["metrics/mAP_0.5:0.95"], label="mAP@0.5:0.95")
plt.xlabel("Epoch")
plt.ylabel("mAP")
plt.title("Mean Average Precision")
plt.legend()
plt.show()

In [ ]:
model = YOLO("/path/to/trained/model.pt")

results = model("/content/dataset/images/test/seq101_000001.jpg")

for result in results:
    result.show()
    result.save("output.jpg")

In [ ]:
metrics = model.val()

print(f"mAP@0.5: {metrics.box.map}")
print(f"mAP@0.5:0.95: {metrics.box.map_50_95}")

## Single Object Tracking